In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

#initial population
population = 100
genes = 100
generations = 100

pop = np.random.randint(0,2, size=(population,genes))
print(pop)


In [ ]:
fitness = np.sum(pop,axis=1)
plt.hist(fitness)


In [ ]:
def elite_selection(fitness):
    return fitness.argsort()[-2:][::-1]  

parents = elite_selection(fitness)
print(pop[parents[0]])


In [ ]:
def crossover(parent1, parent2, crossover_rate):
    # children are copies of parents by default
    child1, child2 = parent1.copy(), parent2.copy()  
    # check for recombination
    if random.random() < crossover_rate:
        # select crossover point that is not on the end of the string
        pt = random.randint(1, len(parent1)-2)
        # perform crossover    
        child1 = np.concatenate((parent1[:pt], parent2[pt:]))
        child2 = np.concatenate((parent2[:pt], parent1[pt:]))
    return [child1, child2]

crossover(pop[parents[0]],pop[parents[1]], .5)


In [ ]:
def mutation(individual, mutation_rate):
    for i in range(len(individual)):
        # check for a mutation
        if random.random() < mutation_rate:
            # flip the bit
            individual[i] = 1 - individual[i]
    return individual

mutation(pop[parents[0]], .5)


In [ ]:
def simple_GA(pop, crossover_rate=.5, mutation_rate=.05):
    fitness = np.sum(pop,axis=1) 
    parents = elite_selection(fitness)
    children = np.zeros((population,genes))  
    for i in range(population):
        offspring = crossover(pop[parents[0]],pop[parents[1]], crossover_rate)
        children[i] = mutation(offspring[0],mutation_rate)  
    return children

simple_GA(pop)


In [ ]:
#initial population
pop = np.random.randint(0,2, size=(population,genes))

for i in range(generations):
    pop = simple_GA(pop)
    fitness = np.sum(pop,axis=1)
    plt.hist(fitness)
    plt.show()
    print(f"Generation {i+1}")
    print(f"    Max fitness {np.max(fitness)}")
    print(f"    Min fitness {np.min(fitness)}")
    print(f"    Mean fitness {np.mean(fitness)}")
    print(f"    Std fitness {np.std(fitness)}")


In [11]:
bs_state=[{
        "user_sa": [],
        "user_bs": [],
        "user_unserve": []
    } for _ in range((2))] 
bs_state[1]["user_bs"].append((1,1))
bs_state[1]["user_bs"].append((2,2))
bs_state[1]["user_bs"][1]

(2, 2)

In [5]:
bs_state=[{
        "user_sa": [],
        "user_bs": [],
        "user_unserve": [],
        "user_bs_req":  [],
    } for _ in range((2))]
bs_state.append({"SINR":False})
bs_state[0]["user_bs"].append(1)
bs_state[1]["user_bs"].append(2)
bs_state[0]["user_bs_req"].append(1)
bs_state[0]["user_bs_req"].append(2)
bs_state[0]["user_bs_req"].append(3)
print(bs_state["user_bs"])


TypeError: list indices must be integers or slices, not str

In [20]:
import numpy as np
x = np.array([3, 1, 2,4,5,6,7])
idx = np.argsort(x)
print(idx)
print(idx[-3:])
print(np.isin(10,x))

[1 2 0 3 4 5 6]
[4 5 6]
False


In [3]:
import numpy as np

# 示例数组
A = np.array([1, 2, 3])
B = np.array([1,2,3,4])

# 找到 B 中没有出现在 A 中的元素
difference = np.setdiff1d(B, A)

print("B 中没有出现在 A 中的元素:", difference)


B 中没有出现在 A 中的元素: [4]


In [8]:
import numpy as np

# 原数组
arr = np.array([0, 2, 3, 3, 3, 3, 4])

# 统计每个元素的出现次数
counts = np.bincount(arr)  # 返回数组中每个值的计数，索引为元素值
print(counts)
# 根据原数组生成新数组
result = arr * counts[arr]

print(result)


[1 0 1 4 1]
[ 0  2 12 12 12 12  4]
